In [ ]:
# @title 1. Preparação do Ambiente (Drive, Instalações e Imports)
from google.colab import drive
import os
import shutil
import tarfile
import gzip
import zipfile
import subprocess
from pathlib import Path

# 1. Montar o Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Drive já montado.")

# 2. Instalar dependências (7zip e unrar)
print("\n--- Verificando e Instalando Dependências ---")

# Instalar p7zip-full
if subprocess.run(['which', '7z'], capture_output=True).returncode != 0:
    print("Instalando p7zip-full...")
    subprocess.run(['sudo', 'apt-get', 'update'], capture_output=True)
    subprocess.run(['sudo', 'apt-get', 'install', '-y', 'p7zip-full'], capture_output=True)
else:
    print("p7zip-full já instalado.")

# Instalar unrar
if subprocess.run(['which', 'unrar'], capture_output=True).returncode != 0:
    print("Instalando unrar...")
    subprocess.run(['sudo', 'apt-get', 'install', '-y', 'unrar'], capture_output=True)
else:
    print("unrar já instalado.")

print("Ambiente pronto!")

In [ ]:
# @title 2. Configurações da Extração

# Caminhos Base
BASE_DRIVE_PATH = Path("/content/drive/MyDrive/PD-FAPESP")
OUTPUT_ROOT_DIR = BASE_DRIVE_PATH / "extraidos"
WORK_DIR = Path("/content/work")

# Configuração dos Arquivos (Padrão Google Takeout)
# Exemplo: takeout-20240511T132758Z-001.tgz
PREFIXO_ARQUIVO = "takeout-20240511T132758Z-"
START_NUM = 20  # Número do primeiro arquivo (ex: 20)
END_NUM = 22    # Número do último arquivo (ex: 22)

# Cria o diretório final se não existir
OUTPUT_ROOT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Diretório de Entrada: {BASE_DRIVE_PATH}")
print(f"Diretório de Saída: {OUTPUT_ROOT_DIR}")
print(f"Processando arquivos de {START_NUM:03d} até {END_NUM:03d}")

In [ ]:
# @title 3. Funções de Extração (Core)

def extract_archive(archive_path, extract_to):
    """Identifica o formato e extrai o arquivo para o diretório alvo."""
    extract_to.mkdir(parents=True, exist_ok=True)
    file_name = archive_path.name
    suffixes = archive_path.suffixes

    try:
        # .tar.gz ou .tgz
        if suffixes[-2:] == ['.tar', '.gz'] or archive_path.suffix == '.tgz':
            print(f"  [TAR.GZ] Extraindo: {file_name}")
            with tarfile.open(archive_path, 'r:*') as tar:
                tar.extractall(extract_to, filter='data') # filter='data' para segurança em Pythons novos

        # .gz (apenas arquivo comprimido, não tar)
        elif archive_path.suffix == '.gz':
            print(f"  [GZ] Extraindo: {file_name}")
            output_file = extract_to / archive_path.stem
            with gzip.open(archive_path, 'rb') as f_in:
                with open(output_file, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

        # .tar
        elif archive_path.suffix == '.tar':
            print(f"  [TAR] Extraindo: {file_name}")
            with tarfile.open(archive_path, 'r') as tar:
                tar.extractall(extract_to, filter='data')

        # .zip
        elif archive_path.suffix == '.zip':
            print(f"  [ZIP] Extraindo: {file_name}")
            with zipfile.ZipFile(archive_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)

        # .7z
        elif archive_path.suffix == '.7z':
            print(f"  [7Z] Extraindo: {file_name}")
            subprocess.run(['7z', 'x', str(archive_path), f'-o{extract_to}', '-aoa'],
                           capture_output=True, check=True)

        # .rar
        elif archive_path.suffix == '.rar':
            print(f"  [RAR] Extraindo: {file_name}")
            subprocess.run(['unrar', 'x', '-o+', str(archive_path), str(extract_to)],
                           capture_output=True, check=True)
        
        else:
            print(f"  [SKIP] Formato não suportado ou arquivo comum: {file_name}")
            return False # Não extraiu nada

        return True # Sucesso

    except Exception as e:
        print(f"  [ERRO] Falha ao extrair {file_name}: {e}")
        return False

def recursive_unpack_and_move(start_file, work_dir, final_dest_dir):
    """
    Copia o arquivo inicial para uma pasta temporária, extrai tudo recursivamente
    (extrai arquivos dentro de arquivos) e move o resultado limpo para o Drive.
    """
    # 1. Limpar e preparar Work Dir
    if work_dir.exists():
        shutil.rmtree(work_dir)
    work_dir.mkdir(parents=True, exist_ok=True)

    print(f"-> Copiando {start_file.name} para área de trabalho temporária...")
    shutil.copy(start_file, work_dir / start_file.name)

    # 2. Extração Recursiva (Loop até não haver mais arquivos compactados)
    changed = True
    while changed:
        changed = False
        archives_found = []
        
        # Escanear por arquivos compactados
        for root, _, files in os.walk(work_dir):
            for file in files:
                fpath = Path(root) / file
                # Lista de extensões suportadas
                if any(fpath.name.lower().endswith(ext) for ext in ['.tgz', '.tar.gz', '.tar', '.gz', '.zip', '.7z', '.rar']):
                    archives_found.append(fpath)

        if archives_found:
            print(f"  Encontrados {len(archives_found)} arquivos compactados internos. Processando...")
            for archive in archives_found:
                # Extrai no mesmo diretório onde o arquivo está
                success = extract_archive(archive, archive.parent)
                if success:
                    try:
                        archive.unlink() # Deleta o arquivo compactado após extrair para economizar espaço
                        changed = True
                    except OSError as e:
                        print(f"    Aviso: Não foi possível deletar {archive.name}: {e}")
        else:
            print("  Nenhum arquivo compactado restante para processar.")

    # 3. Mover arquivos finais para o Google Drive
    print(f"-> Movendo arquivos finais para: {final_dest_dir}")
    final_dest_dir.mkdir(parents=True, exist_ok=True)
    
    files_moved = 0
    for root, _, files in os.walk(work_dir):
        relative_path = Path(root).relative_to(work_dir)
        dest_folder = final_dest_dir / relative_path
        dest_folder.mkdir(parents=True, exist_ok=True)
        
        for file in files:
            src = Path(root) / file
            dest = dest_folder / file
            try:
                shutil.move(src, dest)
                files_moved += 1
            except Exception as e:
                print(f"  Erro ao mover {file}: {e}")

    # 4. Limpeza Final
    if work_dir.exists():
        shutil.rmtree(work_dir)
    
    print(f"Concluído. {files_moved} arquivos movidos.")

In [ ]:
# @title 4. Executar Processamento

for i in range(START_NUM, END_NUM + 1):
    tgz_filename = f"{PREFIXO_ARQUIVO}{i:03d}.tgz"
    source_file = BASE_DRIVE_PATH / tgz_filename
    
    # Define uma subpasta para cada parte extraída para manter organização
    part_output_dir = OUTPUT_ROOT_DIR / f"takeout_part_{i:03d}"

    print(f"\n{'='*10} Iniciando {tgz_filename} {'='*10}")
    
    if not source_file.exists():
        print(f"AVISO: Arquivo não encontrado: {source_file}. Pulando...")
        continue

    recursive_unpack_and_move(source_file, WORK_DIR, part_output_dir)
    
print("\nProcessamento de lote finalizado.")

In [ ]:
# @title 5. Conferência dos Arquivos Extraídos

print(f"Verificando arquivos em: {OUTPUT_ROOT_DIR}\n")
all_files = list(OUTPUT_ROOT_DIR.rglob("*"))
file_count = len([f for f in all_files if f.is_file()])

print(f"Total de arquivos extraídos (em todas as pastas): {file_count}")

print("\nExemplo dos últimos 20 arquivos encontrados:")
for f in all_files[-20:]:
    if f.is_file():
        print(f.relative_to(OUTPUT_ROOT_DIR))